In [42]:
# !pip install selenium
# !pip install webdriver-manager

In [43]:
import time

import pandas as pd
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By

In [44]:
# criado objeto para controle do navegador
edge = webdriver.Edge(service=Service('msedgedriver.exe'), options=Options())

In [45]:
# fazendo request no site
edge.get('https://www.etf.com/etfanalytics/etf-finder')

In [46]:
# clicando no botão de ampliar o display
time.sleep(10)
button_100_xpath = '/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/'
button_100_xpath += 'section[2]/div[2]/section[2]/div[1]/div/div[4]/button/label/span'

button_100 = edge.find_element(By.XPATH, button_100_xpath)
edge.execute_script('arguments[0].click();', button_100)

In [47]:
# pegando o numero total de paginas
total_pages = edge.find_element(By.XPATH, '//*[@id="totalPages"]')
total_pages = int(total_pages.text.replace('of ', ''))
total_pages

32

In [48]:
# pegando tabelas de cada pagina do fund basics
def get_tables():
    
    tables = []
    for page in range(1, total_pages + 1):
        time.sleep(.1)
        table_element = edge.find_element(By.XPATH, '//*[@id="finderTable"]')
        table_html = table_element.get_attribute('outerHTML')
        table = pd.read_html(str(table_html))[0]
        tables.append(table)
    
        next_page_button = edge.find_element(By.XPATH, '//*[@id="nextPage"]')
        edge.execute_script('arguments[0].click();', next_page_button)

    return pd.concat(tables)


etf_tables = get_tables()
etf_tables

,Ticker,Name,Segment,Issuer,Expense Ratio,AUM
0,SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$374.42B
1,IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$306.66B
2,VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$287.12B
3,VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$282.79B
4,QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$172.68B
...,...,...,...,...,...,...
20,CVLC,Calvert US Large-Cap Core Responsible Index ETF,Equity: U.S. - Large Cap,Morgan Stanley,0.15%,--
21,CVMC,Calvert US Mid-Cap Core Responsible Index ETF,Equity: U.S. - Mid Cap,Morgan Stanley,0.15%,--
22,CVIE,Calvert International Responsible Index ETF,Equity: Developed Markets Ex-U.S. - Large Cap,Morgan Stanley,0.18%,--
23,CDEI,"Calvert US Large-Cap Diversity, Equity and Inc...",Equity: U.S. - Large Cap,Morgan Stanley,0.14%,--


In [49]:
# alterando para as tableas de performance
display_pages = edge.find_element(By.XPATH, '//*[@id="goToPage"]')
display_pages.clear()
display_pages.send_keys('1')
display_pages.send_keys(u'\ue007')

performance_table_xpath = '/html/body/div[5]/section/div/div[3]/section/div/div/div'
performance_table_xpath += '/div/div[2]/section[2]/div[2]/ul/li[2]/span'

performance_table_element = edge.find_element(By.XPATH, performance_table_xpath)
edge.execute_script('arguments[0].click();', performance_table_element)


In [50]:
# pegandos as tabelas de performance
etf_performance_table = get_tables()
etf_performance_table.head()

,Ticker,Name,1 Month,3 Month,YTD,1 Year,3 Years,5 Years,10 Years,As Of Date
0,SPY,SPDR S&P 500 ETF Trust,1.50%,6.90%,7.27%,-9.01%,19.91%,11.05%,12.19%,04/04/23
1,IVV,iShares Core S&P 500 ETF,1.49%,6.91%,7.27%,-9.02%,19.97%,11.06%,12.24%,04/04/23
2,VOO,Vanguard S&P 500 ETF,1.53%,6.94%,7.29%,-9.02%,19.98%,11.07%,12.24%,04/04/23
3,VTI,Vanguard Total Stock Market ETF,0.25%,6.24%,6.75%,-10.30%,19.93%,10.25%,11.74%,04/04/23
4,QQQ,Invesco QQQ Trust,6.63%,20.25%,20.01%,-12.94%,21.03%,15.65%,17.73%,04/04/23


In [51]:
# juntando os dados em uma unica tabela
etf_performance_table = etf_performance_table.set_index("Ticker")
etf_tables = etf_tables.set_index("Ticker")

etf_performance_table = etf_performance_table[['1 Year', '3 Years', '5 Years']]

final_db = etf_tables.join(etf_performance_table, how='inner')
final_db.head()

,Name,Segment,Issuer,Expense Ratio,AUM,1 Year,3 Years,5 Years
Ticker,,,,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$374.42B,-9.01%,19.91%,11.05%
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$306.66B,-9.02%,19.97%,11.06%
VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$287.12B,-9.02%,19.98%,11.07%
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$282.79B,-10.30%,19.93%,10.25%
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$172.68B,-12.94%,21.03%,15.65%
